# Coleta e tratamento dos datasets

Nessa seção será demonstrada a captura das informações baixadas no Kaggle:

https://www.kaggle.com/unanimad/tweets-from-brasil-presidential-candidates-2018

In [1]:
#IMPORTA LIBRARIES
import os
import pandas as pd
import numpy as np

In [2]:
#local dos datasets
folder_name = "D:\\PUC_MINAS\\TCC\\Dataset\\tweets"

#função para listar aquivos
def lista_arquivos():
    list_files = []
    for file in os.listdir(folder_name):
        list_files.append(file)
    return list_files   

#recebe todos as linhas
files = lista_arquivos()

In [3]:
#lista de datasets utilizados na pesquisa
print('Lista de datasets para a pesquisa: \n {}\n'.format(files))
print('Total de {} arquivos importados.'.format(len(files)))

Lista de datasets para a pesquisa: 
 ['alvarodias__election_tweets.csv', 'cabodaciolo_election_tweets.csv', 'cirogomes_election_tweets.csv', 'eymaeloficial_election_tweets.csv', 'geraldoalckmin_election_tweets.csv', 'guilhermeboulos_election_tweets.csv', 'haddad_fernando_election_tweets.csv', 'jairbolsonaro_election_tweets.csv', 'joaoamoedonovo_election_tweets.csv', 'joaogoulart54_election_tweets.csv', 'lulaoficial_election_tweets.csv', 'marinasilva_election_tweets.csv', 'meirelles_election_tweets.csv', 'verapstu_election_tweets.csv']

Total de 14 arquivos importados.


In [4]:
#Os campos são separados por ";". No entanto, existem esses caracteres no campo de mensagem
#Optou-se por usar a biblioteca csv, ao invés do pandas
#importar linhas
import csv
def importa_linhas():
    list_rows = []
    for file in files:
        with open(folder_name + "\\" + file, "r", encoding='utf-8') as f:
            rows = csv.reader(f, delimiter=';')
            for i, row in enumerate(rows):
                no_user = file.split('_')
                row.append(no_user[0])
                if(i > 0):
                    list_rows.append(row)
    return list_rows
rows = importa_linhas()
df = pd.DataFrame(rows, columns = ['id','data_msg','tweet','num_retweets','num_likes','user'])

In [5]:
#Mostra as primeiras linhas do dataset
df.head()

,id,data_msg,tweet,num_retweets,num_likes,user
0,1056291241384644609,2018-10-27 21:07:11,NAÇÃO BRASILEIRA. Conh...,55,839,cabodaciolo
1,1056084752371933184,2018-10-27 07:26:40,Seguirei o caminho da integridade;quando virás...,59,1094,cabodaciolo
2,1053461920639057920,2018-10-20 01:44:28,https://t.co/BvTWDIZdiO,893,6128,cabodaciolo
3,1053461098169622529,2018-10-20 01:41:12,Voto em cédulas JÁ!Glória a Deus https://t.co/...,107,871,cabodaciolo
4,1053071583286386689,2018-10-18 23:53:25,@isamovitz Deus abençoe,0,12,cabodaciolo


In [6]:
#Quantidade de linhas importadas
print('Total de {} linhas importadas.'.format(len(df)))

Total de 12286 linhas importadas.


In [7]:
#verifica se há linhas com dados vazios
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12286 entries, 0 to 12285
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            12286 non-null  object
 1   data_msg      12286 non-null  object
 2   tweet         12286 non-null  object
 3   num_retweets  12286 non-null  object
 4   num_likes     12286 non-null  object
 5   user          12286 non-null  object
dtypes: object(6)
memory usage: 576.0+ KB


In [8]:
#verifica se há dados duplicados
df.duplicated().value_counts()

False    12286
dtype: int64

### Foi tratada a coluna created_at para poder trabalhar com o tipo data

In [9]:
def dia_da_semana(week_day):
    week_days = ['Sunday','Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
    l_dias_semana = ['Dom','Seg','Ter','Qua','Qui','Sex','Sab']
    dias_semana = []
    for wd in week_day:
        idx = 0
        for w in week_days:
            if(w == wd):
                dias_semana.append(l_dias_semana[idx])
            idx = idx + 1
    return dias_semana

In [10]:
def data_resumida(data):
    data_resumo = []
    for d in data:
        dt = d.split(' ')
        dt = dt[0].split('-')
        dt = dt[2] + '/' + dt[1]
        data_resumo.append(dt)
    return data_resumo

In [11]:
#Tratar data e hora
df = df.copy()
#Novas colunas
df.loc[:, 'data'] = df.data_msg.astype('datetime64[D]')
df.loc[:, 'data_resumo'] = data_resumida(df.data_msg)
df.loc[:, 'dia_semana'] = dia_da_semana(df.data.dt.day_name())

### Foi feito um tratamento para retirar referências a links nas mensagens

In [12]:
df.loc[:, 'mensagem'] = df.tweet.apply(lambda x: ' '.join(list([r for r in x.split(' ') if(r[0:4]!='http')])))

### Excluir colunas desnecessárias para o estudo

In [13]:
#Excluir algumas colunas desnecessárias
df.drop(columns=['id','data_msg'], inplace=True)
df.head()

,tweet,num_retweets,num_likes,user,data,data_resumo,dia_semana,mensagem
0,NAÇÃO BRASILEIRA. Conh...,55,839,cabodaciolo,2018-10-27,27/10,Sab,NAÇÃO BRASILEIRA. Conh...
1,Seguirei o caminho da integridade;quando virás...,59,1094,cabodaciolo,2018-10-27,27/10,Sab,Seguirei o caminho da integridade;quando virás...
2,https://t.co/BvTWDIZdiO,893,6128,cabodaciolo,2018-10-20,20/10,Sab,
3,Voto em cédulas JÁ!Glória a Deus https://t.co/...,107,871,cabodaciolo,2018-10-20,20/10,Sab,Voto em cédulas JÁ!Glória a Deus
4,@isamovitz Deus abençoe,0,12,cabodaciolo,2018-10-18,18/10,Qui,@isamovitz Deus abençoe


### Criar novo arquivo após tratamento

In [14]:
df.to_csv('../Dataset/tweet_candidatos_eleicoes_2018.csv', sep=';', index=False)